# Training of the CNN model

In [6]:
# -*- coding: utf-8 -*-

import os
import pandas as pd
import numpy as npimage_path_colorbar
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
import tensorflow as tf

datagen = ImageDataGenerator()
# prepare an iterators for each dataset

print("Started...")

train_it = datagen.flow_from_directory('./data/train/',target_size=(224, 224), class_mode='binary',batch_size=40)
val_it = datagen.flow_from_directory('./data/train/',target_size=(224, 224), class_mode='binary',batch_size=40)
test_it = datagen.flow_from_directory('./data/train/',target_size=(224, 224), class_mode='binary',batch_size=40,shuffle=False)
# ptest_it = datagen.flow_from_directory('./data/simple/p_test/',target_size=(224, 224), class_mode='binary',batch_size=40)

# I think p_test is validation 

print("Dataset generated")

num_rows = 224
num_columns = 224
num_channels = 3
num_labels = 2
# Construct model
model_AA_2 = Sequential() # AA - After Augmentation 2- Validation increased
model_AA_2.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model_AA_2.add(MaxPooling2D(pool_size=2))
model_AA_2.add(Dropout(0.2))

model_AA_2.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model_AA_2.add(MaxPooling2D(pool_size=2))
model_AA_2.add(Dropout(0.2))

model_AA_2.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model_AA_2.add(MaxPooling2D(pool_size=2))
model_AA_2.add(Dropout(0.2))

model_AA_2.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model_AA_2.add(MaxPooling2D(pool_size=2))
model_AA_2.add(Dropout(0.2))
model_AA_2.add(GlobalAveragePooling2D())

model_AA_2.add(Dense(1, activation='sigmoid'))

# Compile the model
model_AA_2.compile(optimizer='adam',
              loss='binary_crossentropy', # Loss
              metrics=['binary_accuracy'])

# Display model architecture summary
model_AA_2.summary()

from keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 20
num_batch_size = 40

checkpointer = ModelCheckpoint(filepath='./data/simple/model_AA_2.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()

model_AA_2.fit(train_it,steps_per_epoch=1,epochs = 10, validation_data=val_it,validation_steps= 40,callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

loss = model_AA_2.evaluate(test_it)

model_AA_2.save('./data/model_AA_new.keras')
print("Model Saved")

Started...
Found 23 images belonging to 2 classes.
Found 23 images belonging to 2 classes.
Found 23 images belonging to 2 classes.
Dataset generated
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 223, 223, 16)      208       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 110, 110, 32)      2080      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 55, 55, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 5

In [11]:
# import keras
# import tensorflow as tf
# model = tf.keras.models.load_model('./data/simple/model1.keras')

In [127]:
import keras
import tensorflow as tf
noisy_model = tf.keras.models.load_model('models/model_noisy.keras')

In [128]:
from keras.models import load_model
from keras.preprocessing import image
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
#     img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [131]:
# img_path = 'data/train/test_noisy/3220354.png'
# img_path = 'data/train/test_accepted/3220357.png'
img_path = 'data/train/test_accepted/3220683.png'
# img_path = 'data/train/test_noisy/3220397.png'
# img_path = 'data/train/test_noisy/3220403.png'
# img_path = 'data/train/test_noisy/3220425.png'
# img_path = 'data/train/test_noisy/3220685.png'
# img_path = 'data/train/test_noisy/3220767.png'
new_image = load_image(img_path)

In [132]:
# from keras.preprocessing import image

# test_image = image.load_img(img_path, target_size = (64, 64)) 
# test_image = image.img_to_array(test_image)
# test_image = np.expand_dims(test_image, axis = 0)

In [133]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# datagen = ImageDataGenerator()
# test_it = datagen.flow_from_directory('./data/train/',target_size=(224, 224), class_mode='binary',batch_size=40)

In [134]:
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
print("Start")
# Y_pred = model_AA_2.predict(new_image, verbose = 1, batch_size = 40)
Y_pred = noisy_model.predict(new_image, verbose = 1, batch_size = 40)
print("Done")
print(Y_pred)
y_pred = np.where(Y_pred > 0.5, 1,0)
print(y_pred)
# print(confusion_matrix(test_it.classes, y_pred))
# print(accuracy_score(test_it.classes, y_pred))

Start
1/1 [==============================] - 0s 31ms/step
Done
[[0.02154392]]
[[0]]


In [36]:
print(test_it.class_indices)

{'test_accepted': 0, 'test_noisy': 1}


In [19]:
val_it = datagen.flow_from_directory('./data/simple/validation/',target_size=(224, 224), class_mode='binary',batch_size=40,shuffle=False)

Found 2257 images belonging to 2 classes.


In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix
Y_pred = model_AA_2.predict(val_it,verbose = 1,batch_size = 40)
y_pred = np.where(Y_pred > 0.5, 1,0)
print(confusion_matrix(val_it.classes, y_pred))
print(accuracy_score(val_it.classes, y_pred))

57/57 [==============================] - 22s 378ms/step
[[ 956   44]
 [  55 1202]]
0.9561364643331857


In [25]:
train_it.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

# Production Data Output

In [17]:
prod_Data = datagen.flow_from_directory('./data/prod_data/',target_size=(224, 224), class_mode='binary',batch_size=40,shuffle=False)

Found 1846 images belonging to 2 classes.


In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix
Y_pred = model.predict(prod_Data,verbose = 1,batch_size = 40)
y_pred = np.where(Y_pred > 0.5, 1,0)
print(confusion_matrix(prod_Data.2, y_pred))
print(accuracy_score(prod_Data.classes, y_pred))

47/47 [==============================] - 17s 361ms/step
[[605 241]
 [ 19 981]]
0.8591549295774648


#  Dataset Split Noisy

In [6]:
# import os 
# from tqdm import tqdm

# rdir = "./data/noisy1/simple"

# list = os.listdir(rdir) # dir is your directory path
# number_files = len(list)
# print(number_files)

# train = list[:8000]
# val = list[8000:9000]
# test = list[9000:]

# for file in tqdm(train):
#     os.rename(rdir+"/"+file,"./data/simple/train/noisy"+"/noisy_"+file)

# for file in tqdm(val):
#     os.rename(rdir+"/"+file,"./data/simple/validation/noisy"+"/noisy_"+file)

# for file in tqdm(test):
#     os.rename(rdir+"/"+file,"./data/simple/test/noisy"+"/noisy_"+file)

 33%|███▎      | 2629/8000 [00:00<00:00, 26210.72it/s]

9676


100%|██████████| 676/676 [00:00<00:00, 35397.62it/s]


# Dataset Split Accepted

In [7]:
# rdir = "./data/accepted/simple"

# list = os.listdir(rdir) # dir is your directory path
# number_files = len(list)
# print(number_files)

# train = list[:8000]
# val = list[8000:9000]
# test = list[9000:]

# for file in tqdm(train):
#     os.rename(rdir+"/"+file,"./data/simple/train/accepted"+"/accepted_"+file)

# for file in tqdm(val):
#     os.rename(rdir+"/"+file,"./data/simple/validation/accepted"+"/accepted_"+file)

# for file in tqdm(test):
#     os.rename(rdir+"/"+file,"./data/simple/test/accepted"+"/accepted_"+file)

 33%|███▎      | 2648/8000 [00:00<00:00, 26477.44it/s]

9112


100%|██████████| 112/112 [00:00<00:00, 59037.58it/s]
